In [1]:
import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from catboost import CatBoostClassifier
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, StandardScaler
from sklearn.metrics import auc, accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve

import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from catboost import CatBoostClassifier
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [4]:
class modelDNN(nn.Module):
    def __init__(self):
        super(modelDNN, self).__init__()        
        self.layer_1 = nn.Linear(111, 4096) 
        self.layer_2 = nn.Linear(4096, 2048)
        self.layer_3 = nn.Linear(2048, 64) 
        self.layer_4 = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(4096)
        self.batchnorm2 = nn.BatchNorm1d(2048)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.dropout(x)
        x = self.layer_4(x)
        
        return x

In [5]:
device = torch.device("cuda:0")

In [6]:
model = torch.load("new-KNN.torch")
model.eval()

modelDNN(
  (layer_1): Linear(in_features=111, out_features=4096, bias=True)
  (layer_2): Linear(in_features=4096, out_features=2048, bias=True)
  (layer_3): Linear(in_features=2048, out_features=64, bias=True)
  (layer_4): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [7]:
model

modelDNN(
  (layer_1): Linear(in_features=111, out_features=4096, bias=True)
  (layer_2): Linear(in_features=4096, out_features=2048, bias=True)
  (layer_3): Linear(in_features=2048, out_features=64, bias=True)
  (layer_4): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [8]:
model.layer_4 = nn.Sequential()

In [10]:
model(torch.Tensor(np.ones((10, 111))).to(device)).shape

torch.Size([10, 64])

In [8]:
np.ones((1, 111))

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [2]:
import pickle
f = open("scaler.pckl", 'rb')
scaler = pickle.load(f)
f.close()

In [3]:
def process(df, scaler):
    Y = df.Label
    df = df.drop(['Unnamed: 0','Label'], axis=1)
    df['Attack'] = df['Attack'].astype("string")
    df = df.drop('Attack', axis=1)
    df = df.drop(['SimillarHTTP'], axis=1)
    df[df['Flow Bytes/s'] == np.inf] = 0
    df[df[' Flow Packets/s'] == np.inf] = 0
    df = scaler.transform(df)
    return df, Y

In [4]:
df, Y = process(pd.read_csv('triple.csv'), scaler)

In [8]:
save(df, 'triple.pure')

In [9]:
save(Y, 'triple_Y.pure')

In [7]:
def save(data, filename):
    file = open(filename, 'wb')
    pickle.dump(data, file)
    file.close()

In [54]:
headless_model

Sequential(
  (0): Linear(in_features=111, out_features=4096, bias=True)
  (1): Linear(in_features=4096, out_features=2048, bias=True)
  (2): Linear(in_features=2048, out_features=64, bias=True)
  (3): Linear(in_features=64, out_features=1, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.1, inplace=False)
  (6): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [81]:
chunk=None
chunksize = 10 ** 6

df_x = None
df_y = None
with csv.open_csv(
    "triple.csv",
    convert_options = csv.ConvertOptions(),
    read_options=csv.ReadOptions(
        use_threads=True,
        block_size=chunksize
    )) as reader:

    for next_chunk in reader:
        if next_chunk is None:
            break
        x_t, y_t = process(next_chunk.to_pandas(), scaler)
        if df_x is None:
            df_x, df_y = model(torch.Tensor(x_t).to(device)), y_t
        else:
            torch.cat((df_x, model(torch.Tensor(x_t).to(device))), 0)
            df_y = np.concatenate((df_y, y_t), axis=0)


ArrowInvalid: In CSV column #2: CSV conversion error to int64: invalid value '67834732.0'

In [67]:
model(torch.Tensor(x_t).to(device)).shape

torch.Size([3385, 64])

In [63]:
x_t.shape

(3385, 111)

In [74]:
torch.cat((torch.Tensor(x_t).to(device), torch.Tensor(x_t).to(device)), 0).shape

torch.Size([6770, 111])

In [75]:
torch.Tensor(x_t).to(device).shape

torch.Size([3385, 111])

In [11]:
import pickle

f = open('triple.pure', 'rb')
df = pickle.load(f)
f.close()

In [12]:
model.cpu()

model(torch.tensor(df, dtype=torch.float32))

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 40750661632 bytes. Error code 12 (Cannot allocate memory)

In [17]:
torch.tensor(df, dtype=torch.float32)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1105, 0.5000],
        [0.0000, 0.0000, 0.0000,  ..., 0.1326, 0.1326, 0.5000],
        [0.0000, 0.0000, 0.0000,  ..., 0.1547, 0.1547, 0.5000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [16]:
model.layer_1

Linear(in_features=111, out_features=4096, bias=True)

In [9]:
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [13]:
test_data = TestData(torch.tensor(df, dtype=torch.float32))

In [14]:
loader = DataLoader(dataset=test_data, batch_size=512, shuffle=False)

In [19]:
model.cpu()

f = True
dataset = None
for X_batch in loader:
    if f:
        pd.DataFrame(model(X_batch).detach().numpy()).to_csv("post_knn.csv")
        f = False
    else:
        pd.DataFrame(model(X_batch).detach().numpy()).to_csv("post_knn.csv", mode='a', header=False)

In [30]:
dataset.shape

torch.Size([439, 64])

In [18]:
pd.DataFrame(X_batch.numpy())

,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,110
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99998,...,0.000000,0.516129,0.034483,0.000853,2.899252e-13,0.0,0.106195,0.000000,0.110482,0.5
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99998,...,0.000181,0.774194,0.034483,0.001089,4.501470e-13,0.0,0.106195,0.132574,0.132578,0.5
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99998,...,0.000361,0.774194,0.034483,0.001304,5.900232e-13,0.0,0.106195,0.154670,0.154675,0.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99998,...,0.000542,0.774194,0.034483,0.001515,7.298994e-13,0.0,0.106195,0.176766,0.176771,0.5
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99998,...,0.000903,0.774194,0.034483,0.001764,9.397137e-13,0.0,0.106195,0.220957,0.198868,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99998,...,0.001806,0.774194,0.034483,0.002727,1.593317e-12,0.0,0.106195,0.331436,0.287253,0.5
508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99998,...,0.000000,0.516129,0.034483,0.000853,2.899252e-13,0.0,0.106195,0.000000,0.110482,0.5
509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99998,...,0.000000,0.516129,0.034483,0.000853,2.899252e-13,0.0,0.106195,0.000000,0.110482,0.5
510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99998,...,0.000181,0.774194,0.034483,0.001089,4.501470e-13,0.0,0.106195,0.132574,0.132578,0.5
